# Restaurant Food Cost

# Import libraries

In [94]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

### Import datasets

In [95]:
train = pd.read_excel('Data_Test.xlsx')
test = pd.read_excel('Data_Train.xlsx')

In [96]:
train.shape, test.shape

((4231, 8), (12690, 9))

### Data exploration

In [97]:
train.duplicated().sum(), test.duplicated().sum()

(1, 25)

In [98]:
train.drop_duplicates(keep='first', inplace=True)

In [99]:
train.reset_index(inplace=True)

In [100]:
test.drop_duplicates(keep='first', inplace=True)

In [101]:
test.reset_index(inplace=True)

In [102]:
train.head()

,index,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES
0,0,CASUAL DINING,4085,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564 votes
1,1,QUICK BITES,12680,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61 votes
2,2,CASUAL DINING,1411,"North Indian, Seafood, Biryani, Chinese",11am – 11:30pm (Mon-Sun),Mumbai,Marine Lines,3.8,350 votes
3,3,None,204,Biryani,"9am – 10pm (Mon, Wed, Thu, Fri, Sat, Sun), 10:...",Faridabad,NIT,3.8,1445 votes
4,4,QUICK BITES,13453,"South Indian, Kerala",11am – 10pm (Mon-Sun),Kochi,Kaloor,3.6,23 votes


In [103]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4230 entries, 0 to 4229
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          4230 non-null   int64 
 1   TITLE          4230 non-null   object
 2   RESTAURANT_ID  4230 non-null   int64 
 3   CUISINES       4230 non-null   object
 4   TIME           4230 non-null   object
 5   CITY           4195 non-null   object
 6   LOCALITY       4200 non-null   object
 7   RATING         4228 non-null   object
 8   VOTES          3828 non-null   object
dtypes: int64(2), object(7)
memory usage: 297.5+ KB


In [104]:
train.isnull().sum()

index              0
TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY              35
LOCALITY          30
RATING             2
VOTES            402
dtype: int64

In [105]:
for i in train.columns:
    print("Unique values in", i, train[i].nunique())

Unique values in index 4230
Unique values in TITLE 86
Unique values in RESTAURANT_ID 4127
Unique values in CUISINES 1727
Unique values in TIME 1183
Unique values in CITY 151
Unique values in LOCALITY 834
Unique values in RATING 31
Unique values in VOTES 1136


### Data pre-processing

In [106]:
# merge train and test
df = train.append(test,ignore_index=True)

In [107]:
df = df[['TITLE', 'CUISINES', 'TIME', 'CITY', 'LOCALITY', 'RATING', 'VOTES', 'COST']]

In [108]:
def extract_closed(time):
    a = re.findall('Closed \(.*?\)', time)
    if a != []:
        return a[0]
    else:
        return 'NA'

df['CLOSED'] = df['TIME'].apply(extract_closed)

In [109]:
df['TIME'] = df['TIME'].str.replace(r'Closed \(.*?\)','')

In [110]:
df['TIME'] = df['TIME'].str.replace(r'Closed...','')

In [111]:
df['RATING'] = df['RATING'].str.replace('NEW', '1')
df['RATING'] = df['RATING'].str.replace('-', '1').astype(float)

In [112]:
df['VOTES'] = df['VOTES'].str.replace(' votes', '').astype(float)

In [113]:
df['CITY'].fillna('Missing', inplace=True)  
df['LOCALITY'].fillna('Missing', inplace=True)  
df['RATING'].fillna(3.8, inplace=True)  
df['VOTES'].fillna(0.0, inplace=True) 

In [114]:
df['COST'] = df['COST'].astype(float)

In [115]:
df.head(2)

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST,CLOSED
0,CASUAL DINING,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564.0,NaN,NA
1,QUICK BITES,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61.0,NaN,NA


In [116]:
df['TITLE'].nunique(), df['CUISINES'].nunique()

(123, 5183)

In [117]:
calc_mean = df.groupby(['CITY'], axis=0).agg({'RATING': 'mean'}).reset_index()
calc_mean.columns = ['CITY','CITY_MEAN_RATING']
df = df.merge(calc_mean, on=['CITY'],how='left')

calc_mean = df.groupby(['LOCALITY'], axis=0).agg({'RATING': 'mean'}).reset_index()
calc_mean.columns = ['LOCALITY','LOCALITY_MEAN_RATING']
df = df.merge(calc_mean, on=['LOCALITY'],how='left')

In [118]:
df.head(2)

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST,CLOSED,CITY_MEAN_RATING,LOCALITY_MEAN_RATING
0,CASUAL DINING,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564.0,NaN,NA,3.634672,3.917949
1,QUICK BITES,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61.0,NaN,NA,3.697132,3.723077


In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer


tf1 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_title = tf1.fit_transform(df['TITLE'])
df_title = pd.DataFrame(data=df_title.toarray(), columns=tf1.get_feature_names_out())

tf2 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_cuisines = tf2.fit_transform(df['CUISINES'])
df_cuisines = pd.DataFrame(data=df_cuisines.toarray(), columns=tf2.get_feature_names_out())

tf3 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_city = tf3.fit_transform(df['CITY'])
df_city = pd.DataFrame(data=df_city.toarray(), columns=tf3.get_feature_names_out())

tf4 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_locality = tf4.fit_transform(df['LOCALITY'])
df_locality = pd.DataFrame(data=df_locality.toarray(), columns=tf4.get_feature_names_out())

tf5 = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
df_time = tf5.fit_transform(df['TIME'])
df_time = pd.DataFrame(data=df_time.toarray(), columns=tf5.get_feature_names_out())

In [120]:
df.head(2)

,TITLE,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST,CLOSED,CITY_MEAN_RATING,LOCALITY_MEAN_RATING
0,CASUAL DINING,"North Indian, Chinese, Mughlai, Kebab",12noon – 12midnight (Mon-Sun),Noida,Sector 18,4.3,564.0,NaN,NA,3.634672,3.917949
1,QUICK BITES,"South Indian, Fast Food, Pizza, North Indian",7am – 12:30AM (Mon-Sun),Mumbai,Grant Road,4.2,61.0,NaN,NA,3.697132,3.723077


In [121]:
df = pd.concat([df, df_title, df_cuisines, df_city, df_locality, df_time], axis=1) 
df.drop(['TITLE', 'CUISINES', 'CITY', 'LOCALITY', 'TIME'], axis=1, inplace=True)

In [122]:
df = pd.get_dummies(df, columns=['CLOSED'], drop_first=True)

In [123]:
df.shape

(16895, 2284)

In [124]:
train_df = df[df['COST'].isnull()!=True]
test_df = df[df['COST'].isnull()==True]
test_df.drop('COST', axis=1, inplace=True)

In [125]:
test_df.shape, train_df.shape

((4230, 2283), (12665, 2284))

In [126]:
train_df['COST'] = np.log1p(train_df['COST'])

In [127]:
# Check for duplicate column names
duplicate_columns = df.columns[df.columns.duplicated()]
print("Duplicate feature names:", duplicate_columns)


Duplicate feature names: Index(['bakery', 'bar', 'cafe', 'food', 'paan', 'bar', 'indian', 'kerala',
       'malaysian', 'only',
       ...
       'vyttila', 'wagle', 'ward', 'west', 'whitefield', 'yousufguda', '10',
       '11', '12', '24'],
      dtype='object', length=352)


In [154]:
# Assuming df is your DataFrame and 'bakery' is the duplicate feature name
df.rename(columns={'bakery': 'bakery_2'}, inplace=True)

# Alternatively, you can drop one of the duplicate columns
df.drop(columns=['bakery'], inplace=True)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mdirf\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mdirf\AppData\Local\Temp\ipykernel_11612\3584603512.py", line 5, in <module>
    df.drop(columns=['bakery'], inplace=True)
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    stacklevel=find_stack_level(),
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\core\frame.py", line 5399, in drop
    columns: IndexLabel = ...,
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    stacklevel=find_stack_level(),
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\core\generic.py", line 4505, in drop
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\core\generic.py", line 4546, in _drop_axis
    ...     ],
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pand

In [155]:
try:
    # Your code that may raise an exception goes here
    # For example, training the LightGBM model
    lgbm = lgb.train(params=param, train_set=train_data, valid_sets=[test_data])
    y_pred_lgbm = lgbm.predict(X_cv)
    print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred_lgbm))))
except Exception as e:
    # Print the exception message
    print(f"An error occurred: {str(e)}")
    # Optionally, you can also log the exception for further investigation
    # logging.error(f"An error occurred: {str(e)}")


An error occurred: Feature (bakery) appears more than one time.


In [156]:
print(df.columns.tolist())


['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feature_71', '

In [157]:
# Generate unique names for all features
df.columns = [f"feature_{i}" for i in range(len(df.columns))]


### Train test split

In [158]:
X = train_df.drop(labels=['COST'], axis=1)
y = train_df['COST'].values

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

In [159]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((9498, 2283), (9498,), (3167, 2283), (3167,))

### Build the model

In [164]:
print(type(X_train))
print(type(X_cv))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [165]:
print(X_train.columns)
print(X_cv.columns)

Index(['RATING', 'VOTES', 'CITY_MEAN_RATING', 'LOCALITY_MEAN_RATING', 'bakery',
       'bar', 'beverage', 'bhojanalya', 'bites', 'cafe',
       ...
       'CLOSED_Closed (Mon-Thu)', 'CLOSED_Closed (Mon-Tue)',
       'CLOSED_Closed (Sat)', 'CLOSED_Closed (Sat-Sun)', 'CLOSED_Closed (Sun)',
       'CLOSED_Closed (Thu-Sun)', 'CLOSED_Closed (Tue)', 'CLOSED_Closed (Wed)',
       'CLOSED_Closed (Wed-Sun)', 'CLOSED_NA'],
      dtype='object', length=2283)
Index(['RATING', 'VOTES', 'CITY_MEAN_RATING', 'LOCALITY_MEAN_RATING', 'bakery',
       'bar', 'beverage', 'bhojanalya', 'bites', 'cafe',
       ...
       'CLOSED_Closed (Mon-Thu)', 'CLOSED_Closed (Mon-Tue)',
       'CLOSED_Closed (Sat)', 'CLOSED_Closed (Sat-Sun)', 'CLOSED_Closed (Sun)',
       'CLOSED_Closed (Thu-Sun)', 'CLOSED_Closed (Tue)', 'CLOSED_Closed (Wed)',
       'CLOSED_Closed (Wed-Sun)', 'CLOSED_NA'],
      dtype='object', length=2283)


In [166]:
X_train.columns = X_train.columns.str.strip()
X_cv.columns = X_cv.columns.str.strip()

In [167]:
print(X_train.head())
print(X_cv.head())

       RATING   VOTES  CITY_MEAN_RATING  LOCALITY_MEAN_RATING  bakery  \
16122     3.8  1511.0          3.697132              3.694737     0.0   
16264     4.4  2055.0          3.697132              3.904348     0.0   
7491      3.0     4.0          2.456800              2.405691     0.0   
14972     3.8   318.0          3.634672              3.917949     0.0   
12210     3.5  1211.0          3.621074              3.572222     0.0   

           bar  beverage  bhojanalya     bites  cafe  ...  \
16122  0.80821       0.0         0.0  0.000000   0.0  ...   
16264  1.00000       0.0         0.0  0.000000   0.0  ...   
7491   0.00000       0.0         0.0  0.000000   0.0  ...   
14972  0.00000       0.0         0.0  0.707107   0.0  ...   
12210  0.80821       0.0         0.0  0.000000   0.0  ...   

       CLOSED_Closed (Mon-Thu)  CLOSED_Closed (Mon-Tue)  CLOSED_Closed (Sat)  \
16122                        0                        0                    0   
16264                        0    

In [168]:
categorical_features.append('feature1')

In [175]:
from sklearn.ensemble import BaggingRegressor

br = BaggingRegressor(n_estimators=30, max_samples=0.9, max_features=1.0, bootstrap=True, bootstrap_features=True, oob_score=True, n_jobs=1, random_state=42, verbose=1)

In [180]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=40, criterion='squared_error', max_depth=None, min_samples_split=4, min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, 
                            min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=1, 
                            random_state=42, verbose=1, warm_start=False)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_cv)


In [185]:
# Drop duplicate columns
X_train = X_train.loc[:, ~X_train.columns.duplicated()]

### Predict on test set

In [193]:
Xtrain = train_df.drop(labels='COST', axis=1)

In [194]:
ytrain = train_df['COST'].values

In [195]:
Xtest = test_df

In [196]:
Xtrain.shape, ytrain.shape, Xtest.shape

((12665, 2283), (12665,), (4230, 2283))

In [198]:
X.reset_index(drop=True, inplace=True)


In [205]:
X.loc[train_index]

,RATING,VOTES,CITY_MEAN_RATING,LOCALITY_MEAN_RATING,bakery,bar,beverage,bhojanalya,bites,cafe,...,CLOSED_Closed (Mon-Thu),CLOSED_Closed (Mon-Tue),CLOSED_Closed (Sat),CLOSED_Closed (Sat-Sun),CLOSED_Closed (Sun),CLOSED_Closed (Thu-Sun),CLOSED_Closed (Tue),CLOSED_Closed (Wed),CLOSED_Closed (Wed-Sun),CLOSED_NA
0,3.6,49.0,3.417241,3.457143,0.0,0.00000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
1,4.2,30.0,3.584408,3.472222,0.0,0.80821,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
2,3.8,221.0,3.584408,3.550000,0.0,0.00000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
3,4.1,24.0,3.697132,3.721622,0.0,0.00000,0.0,0.0,0.707107,0.0,...,0,0,0,0,0,0,0,0,0,1
4,3.8,165.0,3.697132,3.986420,0.0,0.00000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660,3.8,546.0,3.621074,3.649074,0.0,0.00000,0.0,0.0,0.707107,0.0,...,0,0,0,0,0,0,0,0,0,1
12661,4.3,1214.0,3.697132,4.175000,0.0,0.80821,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
12662,4.0,608.0,3.519910,3.844231,0.0,0.00000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
12663,3.5,32.0,3.584408,3.487500,0.0,0.00000,0.0,0.0,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1


In [207]:
np.mean(errlgb,0), np.mean(err_br,0), np.mean(err_rf,0)

(nan, nan, nan)

In [208]:
lgbm_final = np.exp(np.mean(y_pred_totlgb,0))
br_final = np.exp(np.mean(y_pred_totbr,0))
rf_final = np.exp(np.mean(y_pred_totrf,0))

In [209]:
y_pred = (lgbm_final*0.70 + br_final*0.215 + rf_final*.15) 
y_pred

nan

In [214]:
df_sub = pd.DataFrame({'COST': [y_pred]})

# Write DataFrame to Excel file
writer = pd.ExcelWriter('Output.xlsx', engine='xlsxwriter')
df_sub.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\mdirf\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mdirf\AppData\Local\Temp\ipykernel_11612\1414018269.py", line 4, in <module>
    writer = pd.ExcelWriter('Output.xlsx', engine='xlsxwriter')
  File "C:\Users\mdirf\anaconda3\lib\site-packages\pandas\io\excel\_xlsxwriter.py", line 198, in __init__
ModuleNotFoundError: No module named 'xlsxwriter'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mdirf\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\mdirf\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\mdirf\anaconda3\lib\site-packages\IPython\co